In [66]:
import cdflib
import pandas as pd
import numpy as np
from matplotlib import colormaps
import matplotlib.lines as pltline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from kamodo_ccmc.flythrough.utils import ConvertCoord
from sgp4.api import Satrec, jday, days2mdhms
from datetime import datetime, timezone, timedelta
import re
from pylab import rcParams
import xarray as xr
from mpl_toolkits.mplot3d import Axes3D
import json
import configparser
import requests
import time
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args

uriBase                = "https://for-testing-only.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query" 

config = configparser.ConfigParser()
config.read("./SpTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

r_e = 6371.

data = xr.open_dataset("../fgm/fgm.nc")

conj = pd.read_pickle('whi_conj_200.pkl')
conj

,sat-ID,int-index,conj-time,conj-dist
0,4578,122.0,2011-07-27 14:24:06,122.685956
1,25039,122.0,2011-07-27 14:24:59,65.421226
2,20432,122.0,2011-07-27 14:25:50,166.397907
3,16262,122.0,2011-07-27 14:27:34,171.232289
4,30777,123.0,2011-07-29 20:51:22,159.361335
5,28809,123.0,2011-07-29 20:54:21,71.838822
6,6392,125.0,2011-08-03 09:47:12,155.521221
7,20442,125.0,2011-08-03 09:47:50,185.874829
8,11060,127.0,2011-08-07 22:28:02,154.663968
9,26548,127.0,2011-08-07 22:29:10,138.173452


In [47]:
# Function definition for TLE search and select
def findTLE(sat, t1, t2):
    requestFindTLE   = "/class/gp_history/NORAD_CAT_ID/" + str(sat) + "/EPOCH/" + str(t1) + "--" + str(t2) + "/format/json/"
    with requests.Session() as session:
        resp = session.post(uriBase + requestLogin, data = siteCred)
        if resp.status_code != 200:
            raise MyError(resp, "POST fail on login")

        resp = session.get(uriBase + requestCmdAction + requestFindTLE)
        if resp.status_code != 200:
            print(resp)
            raise MyError(resp, "GET fail on request for satellites")

        retData = json.loads(resp.text)   
    session.close()
    tle = []
    for t in retData:
        tle = np.append(tle, [[t['TLE_LINE1']], [t['TLE_LINE2']]])
    return tle

def tle_select(ts_list, time):
    TLE_epoch = []
    for i in range(int(len(ts_list)/2)):
        a = 2000 + int(ts_list[2*i].split()[3][:2])
        b, c, d, e, f = days2mdhms(int(ts_list[2*i].split()[3][:2]), float(ts_list[2*i].split()[3][2:]))
        j_tle = np.sum(jday(a, b, c, d, e, f))
        TLE_epoch = np.append(TLE_epoch, j_tle)
        arg = np.argmin(abs(TLE_epoch - time))

    return ts_list[2*arg:2*(arg+1)]

# Function definition for B-field vector generation from IGRF model
# ONLY TAKES IN VALUES ALREADY IN GSE!
def igrf(time, x, y, z):
    lon_gdz, lat_gdz, alt_gdz = ConvertCoord(time, x/r_e, y/r_e, z/r_e, 'GSE', 'car', 'GDZ', 'sph', verbose=False)[:-1]
    
    # setting up lists
    mag = np.zeros(len(lon_gdz))
    diff_lat = np.zeros(len(lon_gdz))
    diff_lon = np.zeros(len(lon_gdz))
    diff_alt = np.zeros(len(lon_gdz))
    diff_alt_km = 0.001 # in km, i.e. 1 m
    eff_r = alt_gdz + r_e # effective radius
    
    # obtain IGRF field data and generate differentials
    for i in range(len(lon_gdz)):
        r = requests.get('https://geomag.bgs.ac.uk/web_service/GMModels/igrf/13/?latitude=' + str(lat_gdz[i]) + '&longitude=' + str(lon_gdz[i]) + '&altitude=' + str(alt_gdz[i]) + '&date=' + str(conj_time.date()) + '&format=json')
        load_field = json.loads(r.text)['geomagnetic-field-model-result']['field-value']
        vert = load_field['vertical-intensity']['value']
        n_prop = load_field['north-intensity']['value'] / abs(vert)
        e_prop = load_field['east-intensity']['value'] / abs(vert)
        diff_lat[i] = n_prop / (2 * np.pi * eff_r[i] / diff_alt_km) * 360
        diff_lon[i] = e_prop / (2 * np.pi * eff_r[i] * np.cos(lat_gdz[i]/180*np.pi) / diff_alt_km) * 360
        diff_alt[i] = vert / abs(vert) * diff_alt_km
        mag[i] = load_field['total-intensity']['value']
    
    # second position to convert back into GSE
    sec_lat = lat_gdz + diff_lat
    sec_lon = lon_gdz + diff_lon
    
    x_2, y_2, z_2 = ConvertCoord(time, sec_lon, sec_lat, alt_gdz - diff_alt, 'GDZ', 'sph', 'GSE', 'car')[:-1]
    u_gse = x_2*r_e - x
    v_gse = y_2*r_e - y
    w_gse = z_2*r_e - z
    
    field_vec = np.stack((u_gse, v_gse, w_gse), axis=1)
    
    return field_vec

# Function definition for quadratic trajectory fitting
# quadratic fit. x, y and z parametrized by time t
def model(t, a, b, c):
    return a*t**2 + b*t + c

# given a matrix of parameters and a time array, generate the x, y and z values of the curve
def curve_gen(time, pmatrix):
    return model(time, pmatrix[0], pmatrix[1], pmatrix[2])

# calculates the parameters in terms of unix time given an array of parameters. Columns are x, y, z and rows are a, b, c
def parameter_get(s, t_sub, pmatrix):
    a = s**2*pmatrix[0]
    b = s*pmatrix[1] - 2*s**2*t_sub*pmatrix[0]
    c = pmatrix[2] - s*t_sub*pmatrix[1] + s**2*t_sub**2*pmatrix[0]
    return np.array((a, b, c))

# generates the parameters as well as the curve 
def fit_gen(r, s, time, sat_pos, check:bool):
    
    # The time parameter requires some moulding in order to converge to the correct solution. 
    # For example, the unix times are typically too big and hence require a rounded subtraction. 
    # The number to subtract is completed by scaling the time down by 'r' and then forcing it to 
    # be an integer - this is what I refer to by 'rounding'. Then 's' is the scaling. The new
    # time variable is 't_fit'. 
    t_sub = int(time[0]*r)/r
    t_fit = (time - t_sub) * s

    px, ex = curve_fit(model, t_fit, sat_pos[:,0])
    py, ey = curve_fit(model, t_fit, sat_pos[:,1])
    pz, ez = curve_fit(model, t_fit, sat_pos[:,2])
    
    p_fit = np.array((px, py, pz)).T
    
#     p_array = parameter_get(s, t_sub, p_fit)
    
    # Checking vaildity of curve fit. The first ratio should be much smaller than one, 
    # and the condition number should be as small as possible. 
    if check == True: 
        print('Ratio of covariance diagonal to parameter:', np.max(np.diag(ex)/px))
        print('Condition number of covariance matrix:', round(np.max([np.linalg.cond(ex), np.linalg.cond(ey), np.linalg.cond(ez)])))
    
    x_fit = curve_gen(t_fit, px)
    y_fit = curve_gen(t_fit, py)
    z_fit = curve_gen(t_fit, pz)
    
    r_fit = np.stack((x_fit, y_fit, z_fit), axis=1)
    
    return r_fit, p_fit, np.array([t_sub, s]), t_fit

# Function generation for surface-curve intersection search
def is_search(sat_normal, sat_pos, c2_pos, field_vec, buffer):
    find_list = []
    find_c2_index = []
    find_sat_index = []
    mu_dist_list = []
        
    for j in range(len(sat_normal)):
        find_list_n = []
        find_c2_index_n = []
        mu_dist_list_n = []
        field_vec_no = field_vec[j]
        sat_norm_no = sat_normal[j]
        
        for i in range(len(c2_pos)):
            int_vec_diff = c2_pos[i] - sat_pos[j] # vector of intersection - point
            cross = np.cross(field_vec_no, sat_norm_no)
            cross_norm = cross/np.sum(cross**2)**0.5
            mu_dist = abs(np.dot(int_vec_diff, cross_norm))
            
            if mu_dist < buffer:
                find_list_n.append(abs(np.dot(sat_norm_no, c2_pos[i]) - np.dot(sat_norm_no, sat_pos[j])))
                find_c2_index_n.append(i)
                mu_dist_list_n.append(mu_dist)
            else: 
                continue
        
        if len(find_list_n) > 0:
            find_list.append(np.min(find_list_n))
            find_c2_index.append(find_c2_index_n[np.argmin(find_list_n)])
            mu_dist_list.append(mu_dist_list_n[np.argmin(find_list_n)])
            find_sat_index.append(j)
    
    print(len(find_list), 'out of', len(sat_normal))
    
    vec_index = find_sat_index[np.argmin(find_list)]
    c2_pos_index = find_c2_index[np.argmin(find_list)]
    pl_int_t = t[c2_pos_index] # plane-intersection time
    
    return vec_index, c2_pos_index, pl_int_t, mu_dist_list, find_list

In [12]:
buffer = 180 # in seconds

norm_id_list = []
c2_pos_id_list = []
sat_pos_id_list = []
int_time_list = []

print('Total:', len(conj))

Total: 44


#### Loop (run ONCE ONLY)

In [48]:
%%time
for j in range(len(conj)):
    sat_id = conj['sat-ID'].iloc[j]
    conj_time = conj['conj-time'].iloc[j]
    
    sta = conj_time - timedelta(seconds=buffer)
    end = conj_time + timedelta(seconds=buffer)

    sat_time = np.arange(sta.timestamp(), end.timestamp()) # unix time range for coordinate conversions

    # converting times to Julian dates for SGP4 propagation
    j_date = list(map(int, str(conj_time).split()[0].split('-')))
    j_time = list(map(int, str(conj_time).split()[1].split(':')))
    j_conj_time = np.sum(jday(j_date[0], j_date[1], j_date[2], j_time[0], j_time[1], j_time[2]))

    j_sta = j_conj_time - buffer/86400
    j_end = j_conj_time + buffer/86400
    
    # CLUSTER data extraction
    spec = data.sel(t=slice(sta,end))
    maxb = np.max(spec['bvec'].values)*1.1

    t = pd.to_datetime(spec['t'].values)
    t_plot =  np.array([i.value*1e-9 for i in t])
    t1 = pd.to_datetime(spec['t'].values[0]).round('s')
    t2 = pd.to_datetime(spec['t'].values[-1]).round('s')
    d1 = t1.date()
    d2 = t2.date()

    bx = spec['bvec'].values[:,0]
    by = spec['bvec'].values[:,1]
    bz = spec['bvec'].values[:,2]
    bmag = spec['bvec'].values[:,3]

    x = spec['bvec'].values[:,4]
    y = spec['bvec'].values[:,5]
    z = spec['bvec'].values[:,6]
    
    # TLE search for given satellite, trajectory and IGRF B-field vector generation
    # 'tle search' abbreviated as 'ts'
    ts_buffer = 1 # in days
    ts_e1 = d1 - timedelta(days=ts_buffer)
    ts_e2 = d2 + timedelta(days=ts_buffer)

    ts_list = findTLE(sat_id, ts_e1, ts_e2) # list of eligible TLEs
    sat_tle = tle_select(ts_list, j_conj_time) # selecting the closest TLE

    # SGP4 propagation
    j_range = np.arange(j_sta, j_end, 1/86400)
    r_sat = np.zeros((len(j_range),3))
    for i in range(len(j_range)):
        r_sat[i,:] = Satrec.twoline2rv(sat_tle[0], sat_tle[1]).sgp4(j_range[i],0)[1]

    sx_teme = r_sat[:,0]/r_e
    sy_teme = r_sat[:,1]/r_e
    sz_teme = r_sat[:,2]/r_e

    # converting from TEME (TLE) to GSE (CLUSTER)
    sx, sy, sz = ConvertCoord(sat_time, sx_teme, sy_teme, sz_teme, 'teme', 'car', 'GSE', 'car', verbose=False)[:-1]
    sx*=r_e
    sy*=r_e
    sz*=r_e

    sat_pos = np.stack((sx, sy, sz), axis=1)
    
    # Curve fit for satellite and CLUSTER, IGRF B-field vector generation for satellite trajectory
    sat_r_fit, sat_p_array, f_parm = fit_gen(1e-1, 0.005, sat_time, sat_pos, True)[:3]
    sfx, sfy, sfz = sat_r_fit.T

    c2_r_fit, c2_p_array, f_parm = fit_gen(1e-1, 0.005, t_plot, np.stack((x, y, z), axis=1), True)[:3]
    cfx, cfy, cfz = c2_r_fit.T
    
    parm_array = np.concatenate((sat_p_array, c2_p_array), axis=1)
    
    # IGRF B-field vector generation
    sfield_vec = igrf(sat_time, sfx, sfy, sfz)
    
    # field line generation
    alpha = np.full(len(sfx), 100000) # to be changed
    fneg = sat_r_fit - (sfield_vec.T*alpha).T
    fpos = sat_r_fit + (sfield_vec.T*alpha).T
    
    # Field surface parametrisation, normal vector generation, calling surface-curve intersection search
    vec1_list = (sfield_vec.T*alpha).T[:-1] # field vector
    vec2_list = (np.roll(sat_r_fit, -1, axis=0) - sat_r_fit)[:-1] # velocity vector

    sat_normal = np.zeros((1, 3))
    for i in range(len(vec1_list)):
        vec1 = vec1_list[i,:]
        vec2 = vec2_list[i,:]
        sat_normal = np.append(sat_normal, np.resize(np.cross(vec1, vec2),(1,3)),axis=0)
    sat_normal = sat_normal[1:]

    norm_id, c2_pos_id, int_time, mu_dist_list, find_list = is_search(sat_normal, sat_r_fit, c2_r_fit, sfield_vec, 5)
    sat_pos_id = np.argmin(abs(sat_time - int(pd.to_datetime(int_time).round('s').value*1e-9)))
    
    pd.DataFrame(parm_array, columns=['sat_x','sat_y','sat_z','c2_x','c2_y','c2_z'], index=['a','b','c']).to_pickle('sat_' + str(sat_id) + '_parm.pkl')
    sat_dt = pd.to_datetime(sat_time*1e9)
    pd.DataFrame({'u': sfield_vec[:,0], 'v': sfield_vec[:,1], 'w': sfield_vec[:,2]}, index=sat_dt).to_pickle('sat_' + str(sat_id) + '_field.pkl')
    pd.DataFrame({'x': sat_normal[:,0], 'y': sat_normal[:,1], 'z': sat_normal[:,2]}, index=sat_dt[:-1]).to_pickle('sat_' + str(sat_id) + '_norm.pkl')
    
    norm_id_list.append(norm_id)
    c2_pos_id_list.append(c2_pos_id)
    sat_pos_id_list.append(sat_pos_id)
    int_time_list.append(int_time)
    print(j)

Ratio of covariance diagonal to parameter: 8.080649803301574e-06
Condition number of covariance matrix: 351
Ratio of covariance diagonal to parameter: 4.996389453652041e-05
Condition number of covariance matrix: 355
0
Ratio of covariance diagonal to parameter: 0.00023918335549017125
Condition number of covariance matrix: 373
Ratio of covariance diagonal to parameter: 5.23850940296089e-05
Condition number of covariance matrix: 376
1
Ratio of covariance diagonal to parameter: 4.1848913155018775e-06
Condition number of covariance matrix: 312
Ratio of covariance diagonal to parameter: 4.6404997694145356e-05
Condition number of covariance matrix: 315
2
Ratio of covariance diagonal to parameter: 0.002372823897714014
Condition number of covariance matrix: 337
Ratio of covariance diagonal to parameter: 3.814388546268225e-05
Condition number of covariance matrix: 341
3
Ratio of covariance diagonal to parameter: 8.563618077587676e-06
Condition number of covariance matrix: 324
Ratio of covariance

Ratio of covariance diagonal to parameter: 0.00013350244769308917
Condition number of covariance matrix: 318
Ratio of covariance diagonal to parameter: 3.7328929416488916e-06
Condition number of covariance matrix: 321
38
Ratio of covariance diagonal to parameter: 0.00013630511151184804
Condition number of covariance matrix: 373
Ratio of covariance diagonal to parameter: 2.721993016942573e-06
Condition number of covariance matrix: 365
39
Ratio of covariance diagonal to parameter: 1.8869320751484663e-05
Condition number of covariance matrix: 358
Ratio of covariance diagonal to parameter: 5.214484828772191e-06
Condition number of covariance matrix: 336
40
Ratio of covariance diagonal to parameter: 5.232692048540055e-06
Condition number of covariance matrix: 373
Ratio of covariance diagonal to parameter: 1.8424182282326748e-06
Condition number of covariance matrix: 376
41
Ratio of covariance diagonal to parameter: 0.000128899750786356
Condition number of covariance matrix: 324
Ratio of cov

In [23]:
pd.DataFrame({'conj_dist': [round(i,1) for i in conj['conj-dist'].values], 'conj_time': conj['conj-time'].values, 'int_time': int_time_list, 'c2_pos_id': c2_pos_id_list, 'sat_pos_id': sat_pos_id_list, 'sat_field_id': norm_id_list}, index=conj['sat-ID']).to_pickle('intersection_list.pkl')

In [24]:
pd.read_pickle('intersection_list.pkl')

,conj_dist,conj_time,int_time,c2_pos_id,sat_pos_id,sat_field_id
sat-ID,,,,,,
4578,122.7,2011-07-27 14:24:06,2011-07-27 14:23:56.300,851,170,148
25039,65.4,2011-07-27 14:24:59,2011-07-27 14:24:34.300,776,155,173
20432,166.4,2011-07-27 14:25:50,2011-07-27 14:25:08.900,694,139,217
16262,171.2,2011-07-27 14:27:34,2011-07-27 14:27:00.900,734,147,200
30777,159.4,2011-07-29 20:51:22,2011-07-29 20:51:27.700,928,186,145
28809,71.8,2011-07-29 20:54:21,2011-07-29 20:54:04.900,819,164,182
6392,155.5,2011-08-03 09:47:12,2011-08-03 09:47:05.300,866,173,162
20442,185.9,2011-08-03 09:47:50,2011-08-03 09:47:32.300,811,162,204
11060,154.7,2011-08-07 22:28:02,2011-08-07 22:29:57.500,1477,296,199
